In [1]:
import json
from pymongo import MongoClient
from googletrans import Translator
from bson.objectid import ObjectId
import tqdm
import re
from nltk import ngrams
import Levenshtein
import pickle
import os
from gensim.test.utils import lee_corpus_list
from gensim.models import Word2Vec, Doc2Vec
import json
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
client = MongoClient(host='103.252.1.144', username='admin', password = 'CIST2o20')
cb = client['human_resources']['career_builder']
cv = cb.find({'eng_experience':{'$exists':True}, 'fulltext':{'$regex':'.*developer.*'}})
jpt = client['human_resources']['job_posting_topcv']
jd = jpt.find({'eng_job_description':{'$regex':'.*developer.*'}})

In [3]:
cb = client['human_resources']['linkedin_skills']
skill = cb.find()

In [4]:
skill = cb.find()
cnt=0
for i in skill:
    if i['explore_topics']:
        cnt+=1

In [7]:
skill = cb.find()
graph = {}

for i in skill:
    if i['explore_topics']!=None:
        graph[i['skill'].lower()] = [k.lower() for k in i['explore_topics']] 

In [12]:
skill = cb.find()
dict_software = {}
for i in skill:
    # else:
    #     graph[i['skill'].lower()] = []
    if i['skill'].lower() == 'business analysis'.lower():
        print(i['skill'])
        print(i['software'])
        print(i['explore_topics'])
    dict_software[i['skill'].lower()] = [m.lower() for m in i['software']]

Business Analysis
['Microsoft Excel', 'Connections', 'FileMaker Pro', 'NASBA Continuing Professional Education (CPE)', 'Microsoft Office', 'Nintex', 'SharePoint', 'Academic Credits', 'PowerPoint']
None


In [9]:
graph['business analysis and strategy']

['business analysis',
 'business intelligence',
 'business strategy',
 'crisis management',
 'data analysis',
 'data visualization',
 'ethics and law',
 'operations management',
 'product management',
 'supply chain management']

In [11]:
domain = set()
for i in graph:
    for j in graph[i]:
        if j in graph:
            domain.add(i)

for i in {'Databases', 'Finance', 'YAML'}:
    domain.add(i.lower())

In [16]:
graph_ = {}
for i in domain:
    graph_[i.lower()] = {}
    for j in graph[i]:
        graph_[i.lower()][j.lower()] = {}
        if j in graph:
            graph_[i.lower()][j.lower()] = {}
            for k in graph[j]:
                graph_[i.lower()][j.lower()][k]={}


# for i in graph_:
#     if i in dict_software:
#         graph_[i]['Software'] = {}
#         for k in dict_software[i]:    
#             graph_[i]['Software'][k] = {}
#     for j in graph_[i]:
#         if j in dict_software:
#             graph_[i][j]={}
#             for k in dict_software[j]:               
#                 graph_[i][j][k] = {}

In [23]:
for i in graph_:
    if i in dict_software:
        graph_[i]['Software_of_'+i] = {}
        for k in dict_software[i]:    
            graph_[i]['Software_of_'+i][k] = {}
    for j in graph_[i]:
        if j in dict_software:
            graph_[i][j]['Software_of_'+j] = {}
            for k in dict_software[j]:    
                graph_[i][j]['Software_of_'+j][k] = {}
        for m in graph_[i][j]:
            if m in dict_software:
                graph_[i][j][m]={}
                for k in dict_software[m]:               
                    graph_[i][j][m][k] = {}

In [36]:
graph_['business']['business analysis and strategy'].keys()

dict_keys(['business analysis', 'business intelligence', 'business strategy', 'crisis management', 'data analysis', 'data visualization', 'ethics and law', 'operations management', 'product management', 'supply chain management', 'Software_of_business analysis and strategy'])

In [25]:
for i in graph_:
    for j in graph_[i]:
        if j == 'animation and illustration':
            print(graph_[i][j])

{'2d animation': {'after effects': {}, 'cinema 4d': {}, 'maya': {}, 'blender': {}, '3ds max': {}, 'photoshop': {}, 'unreal engine': {}, 'houdini': {}, 'harmony': {}, 'premiere pro': {}, 'adobe animate': {}, 'solidworks': {}, 'illustrator': {}, 'indesign': {}, 'touchdesigner': {}, 'unity': {}, 'motion': {}, 'flash professional': {}, 'python': {}, 'css': {}}, '3d animation': {'3ds max': {}, 'maya': {}, 'cinema 4d': {}, 'blender': {}, 'after effects': {}, 'v-ray': {}, 'houdini': {}, 'unreal engine': {}, 'krita': {}, 'substance designer': {}, 'modo': {}, 'zbrush': {}, 'solidworks': {}, 'unity': {}, 'premiere pro': {}, 'adobe animate': {}, 'python': {}, 'css': {}, 'javascript': {}, 'illustrator': {}}, '3d lighting': {'3ds max': {}, 'maya': {}, 'v-ray': {}, 'substance painter': {}, 'substance designer': {}, 'after effects': {}, 'unreal engine': {}}, '3d materials': {'maya': {}, 'after effects': {}, 'photoshop': {}, 'substance painter': {}, '3ds max': {}, 'marvelous designer': {}, 'zbrush': {

In [26]:
graph_.keys()

dict_keys(['finance', 'business', 'databases', 'yaml', 'creative', 'technology'])

In [53]:
import numpy as np
for i in graph_:
    cnt = []
    for j in graph_[i]:
        for k in graph_[i][j]:
            cnt.append(len(graph_[i][j][k]))
        # cnt.append(np.mean(cnt_k))
        
    print(i, end=': ')
    print(np.mean(cnt), end=' child level_3 skills\n')

finance: 6.88 child level_3 skills
business: 9.476190476190476 child level_3 skills
databases: 10.416666666666666 child level_3 skills
yaml: 13.08108108108108 child level_3 skills
creative: 10.806763285024154 child level_3 skills
technology: 15.2 child level_3 skills


In [28]:
graph_['business']['business analysis and strategy']

{'business analysis': {'microsoft excel': {},
  'connections': {},
  'filemaker pro': {},
  'nasba continuing professional education (cpe)': {},
  'microsoft office': {},
  'nintex': {},
  'sharepoint': {},
  'academic credits': {},
  'powerpoint': {}},
 'business intelligence': {'power bi': {},
  'microsoft excel': {},
  'sap erp': {},
  'microsoft office': {},
  'office 365': {},
  'powerapps': {},
  'power automate': {},
  'tableau': {},
  'power bi for office 365': {},
  'power apps': {},
  'dynamics': {},
  'sharepoint online': {},
  'microsoft 365': {},
  'r': {},
  'nasba continuing professional education (cpe)': {},
  'python': {},
  'amazon web services': {},
  'finance erp': {},
  'stata': {},
  'google analytics': {}},
 'business strategy': {'python': {},
  'office 365': {},
  'microsoft excel': {},
  'hr certification institute® (hrci®)': {},
  'society for human resource management (shrm®)': {},
  'project management institute (pmi)®': {},
  'powershell': {},
  'nasba cont

In [30]:
graph_.keys()

dict_keys(['finance', 'business', 'databases', 'yaml', 'creative', 'technology'])

In [31]:
f = open('ontology.json', 'r')
g_onto = json.loads(f.read())

In [32]:
g_onto['linked_in'] = graph_

In [33]:
g_onto_o = {}
g_onto_o['all'] = g_onto

In [34]:
with open('ontology_linked_in_general.json', 'w') as f:
    json.dump(g_onto_o, f)

In [35]:
g_onto_o['all'].keys()

dict_keys(['general', 'linked_in'])

In [184]:
g_onto_o['all'].keys()

dict_keys(['general', 'linked_in'])

In [193]:
g_onto_o['all']['linked_in']['technology']

{'cloud computing': [],
 'data science': ['artificial intelligence (ai)',
  'big data',
  'business analytics',
  'business intelligence',
  'cryptocurrency',
  'data analysis',
  'data engineering',
  'data governance',
  'data modeling',
  'data privacy',
  'data science careers',
  'data visualization',
  'gis',
  'machine learning',
  'statistics'],
 'database management': ['data centers',
  'data resource management',
  'database administration',
  'database development'],
 'devops': ['agile software development',
  'devops foundations',
  'devops tools'],
 'it help desk': ['client operating systems',
  'help desk skills',
  'operating system distribution',
  'software support',
  'upgrade and maintenance'],
 'mobile development': ['android development',
  'cross-platform development',
  'mobile game development',
  'ios development'],
 'network and system administration': ['crm administration',
  'enterprise content management',
  'it automation',
  'it service management',
  'in